In [ ]:
from __future__ import print_function
import gym
import gym_kheperaposition
import numpy as np
import math
import sys
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

Wmax=math.pi/4
Kr_Prop=0.025   
K1=0.1
Kp=0.75
Ki=0.00007 
PI=3.1415

Vmax=0.05
Wmax=math.pi/4
L=0.1
Kr_V_RL=0.05   # radio de seguridad
# mostrar el entorno

noDetectionDistance=0.2 

# Parametros para Braintenberg
braitFrontSens_leftMotor=[5,3,2]       
braitFrontSens_rightMotor=[-5,-3,-2]
braitSideSens_leftMotor=[6,-6]
braitFrontLateralSens_leftMotor=[4,-4]

def BraitenbergControl(v,w,S):

    S = [(abs(sensor-1)*0.2)  for sensor in S]
    Vri=(2*v+w*0.1)/2     
    Vle=(2*v-w*0.1)/2

    if S[3]==noDetectionDistance:
        if S[2]+S[4]+S[1]+S[5]<=noDetectionDistance*4:
            if S[1]>0.25*noDetectionDistance:
                Vle = Vle + Vmax*braitSideSens_leftMotor[0]*(1-(S[1]/noDetectionDistance))                   
                Vri = Vri + Vmax*braitSideSens_leftMotor[1]*(1-(S[1]/noDetectionDistance))
            if S[2]>0.25*noDetectionDistance:
                Vle = Vle + Vmax*braitFrontLateralSens_leftMotor[0]*(1-(S[2]/noDetectionDistance))                    
                Vri = Vri + Vmax*braitFrontLateralSens_leftMotor[1]*(1-(S[2]/noDetectionDistance))            
            if S[4]>0.25*noDetectionDistance:
                Vle = Vle + Vmax*braitFrontLateralSens_leftMotor[1]*(1-(S[4]/noDetectionDistance))                    
                Vri = Vri + Vmax*braitFrontLateralSens_leftMotor[0]*(1-(S[4]/noDetectionDistance))                
            if S[5]>0.25*noDetectionDistance:
                Vle = Vle + Vmax*braitSideSens_leftMotor[1]*(1-(S[5]/noDetectionDistance))
                Vri = Vri + Vmax*braitSideSens_leftMotor[0]*(1-(S[5]/noDetectionDistance))
    else:
        for i in range(3):
            Vle = Vle + Vmax*braitFrontSens_leftMotor[i]*(1-(S[i+2]/noDetectionDistance))
            Vri = Vri + Vmax*braitFrontSens_rightMotor[i]*(1-(S[i+2]/noDetectionDistance))
            
    lineal=(Vri+Vle)/2
    angular=(Vri-Vle)/0.1
        
    if angular>Wmax:
        angular=Wmax
    if angular<-Wmax:
        angular=-Wmax

    return(lineal,angular) 
 

def Villela_Braitenberg_agent(steps=1100, angle=0):
    episode_reward = 0
    ErrorAcumulado=0
    env = gym.make('KheperaPositionObstacle-v0')
    env.MaxSteps = steps
    env.Randomize = False
    env.RobotOrientationRand = False
    env.radius = 0.95
    #x,y = env.change_target_angle(angle)
    #env.change_target_angle(45)
    state = env.reset()
    d,Oc,vp,wp,s1,s2,s3,s4,s5,s6,s7,s8 = state
    sensors = [s1,s2,s3,s4,s5,s6,s7,s8]
    #env.render()
    ISE_acumulado = 0
    IAE_acumulado = 0
    ITSE_acumulado = 0
    ITAE_acumulado = 0
    
    data = {}
    paso = []
    infoOc = []
    infoD = []
    Lineal = []
    Angular = []
    xc = []
    yc = []

    
    for e in range(steps):
        
        w = Wmax*math.sin(Oc)
        if d > Kr_V_RL:
            v = Vmax
        else:
            v = d*(Vmax/Kr_V_RL)
        if d < 0.02:
            v = 0
            w = 0
        v,w = BraitenbergControl(v,w,sensors)
        
        if d < Kr_Prop:
            v = 0
            w = 0
        
        v = v/Vmax
        w = w/Wmax
        
        action = [v,w]
        state, reward, done, info = env.step(action)
        d,Oc,vp,wp,s1,s2,s3,s4,s5,s6,s7,s8 = state
        sensors = [s1,s2,s3,s4,s5,s6,s7,s8]
        #env.render()
        #print(reward)
        print(reward, end='\r')
        sys.stdout.flush()

        episode_reward += reward
        x = info['xc']
        y = info['yc']
        
        paso.append(e)
        infoOc.append(float(Oc))
        infoD.append(float(d))
        Lineal.append(float(v))
        Angular.append(float(w))
        xc.append(float(x))
        yc.append(float(y))
             
        if done:
            print('Reward of the episode is: ',episode_reward)
            break
    env.close()
   
    data={'paso' : paso,
          'Oc' : infoOc,
          'Distance' : infoD,
          'Lineal' : Lineal,
          'Angular' : Angular,
          'xc' : xc,
          'yc' : yc}
    
    #print(data)
    return data

agent = Villela_Braitenberg_agent(steps=2000)